In [1]:
# pip install pyarrow

In [2]:
# pip install awscli

In [5]:
# import os
# !set AWS_SHARED_CREDENTIALS_FILE=awscli.ini
# path = "awscli.ini"
# os.environ['AWS_SHARED_CREDENTIALS_FILE'] = path

# !aws configure list

      Name                    Value             Type    Location
      ----                    -----             ----    --------
   profile                <not set>             None    None
access_key     ****************ATZI shared-credentials-file    
secret_key     ****************e3kB shared-credentials-file    
    region                us-west-1      config-file    ~/.aws/config


In [6]:
%%time
import pyarrow.dataset as ds
dataset = ds.dataset("s3://ursa-labs-taxi-data/", partitioning=["year", "month"])
dataset

CPU times: total: 1.56 s
Wall time: 21.9 s


In [19]:
%%time
# * Which time (hour) of the day has the highest tip?

import pyarrow.compute as pc
import pyarrow.parquet as pq
from datetime import datetime
hour_count = [0] * 24
hour_tips = [0] * 24
hour_average = [0] * 24 
columns_to_select =['dropoff_at', 'tip_amount']
for frag in dataset.get_fragments():
#     print("New Fragment")
    frag_table = frag.to_table(columns = columns_to_select)
    sorted_indices = pc.sort_indices([dt.as_py().hour for dt in frag_table["dropoff_at"]])
    sorted_table = frag_table.take(sorted_indices)

    row_group_max_size = frag_table.to_batches()[0].num_rows
    pq.write_table(sorted_table, 'optimized_parquet_file.parquet', row_group_size=row_group_max_size)
    optimized_parquet_file = pq.ParquetFile('optimized_parquet_file.parquet')

    for i in range(optimized_parquet_file.num_row_groups):
#         print("New Row Group")
    # name_2_pos = {x["path_in_schema"]:i for i, x in enumerate(optimized_parquet_file.metadata.row_group(i).to_dict()["columns"])}
        col_stats = optimized_parquet_file.metadata.row_group(i).column(0).statistics
#         print("min : ",col_stats.min.hour,"max : ",col_stats.max.hour)
        tip_amounts= optimized_parquet_file.read_row_group(i)["tip_amount"]
        if col_stats.min.hour == col_stats.max.hour:
#             print("IF")
            hour_count[col_stats.min.hour] += len(tip_amounts)
            hour_tips[col_stats.min.hour] += pc.sum(tip_amounts).as_py()
#             print("LEN: ",len(tip_amounts))
#             print("SUM : ",pc.sum(tip_amounts).as_py())
        else:
#             print("ELSE")
            dropoffs= optimized_parquet_file.read_row_group(i)["dropoff_at"]
            for hour in range(col_stats.min.hour,col_stats.max.hour+1):
                filtered_hour = [dropoff_at.as_py().hour == hour for dropoff_at in dropoffs]
                filtered_tip_amount = pc.filter(tip_amounts, filtered_hour)
                if len(filtered_tip_amount)>0 :
#                     print("LEN: ",len(filtered_tip_amount))
#                     print("SUM : ",pc.sum(filtered_tip_amount).as_py())
                    hour_count[hour] += len(filtered_tip_amount)
                    hour_tips[hour] += pc.sum(filtered_tip_amount).as_py()
#         print("hour_count",hour_count)
#         print(" hour_tips", hour_tips)

for hour in range(24):
    if hour_count[hour]>0:
        hour_average[hour] = hour_tips[hour]/hour_count[hour]
        print ("Average tip of ",hour," o'Clock is ", hour_average[hour])
        

max_average_tip, max_hour = max((value, index) for index, value in enumerate(hour_average))
print (max_hour," o'Clock, on average has the highest tip of ",max_average_tip)

Average tip of  0  o'Clock is  1.4243779527112415
Average tip of  1  o'Clock is  1.3442984866228143
Average tip of  2  o'Clock is  1.2371060744676055
Average tip of  3  o'Clock is  1.198042766172848
Average tip of  4  o'Clock is  1.1992213773037776
Average tip of  5  o'Clock is  1.4427898625866893
Average tip of  6  o'Clock is  1.3122225741059355
Average tip of  7  o'Clock is  1.2269625773606119
Average tip of  8  o'Clock is  1.2818543596148426
Average tip of  9  o'Clock is  1.3147329248500712
Average tip of  10  o'Clock is  1.2388282293011472
Average tip of  11  o'Clock is  1.2109359922782343
Average tip of  12  o'Clock is  1.205665626765673
Average tip of  13  o'Clock is  1.2084415345419697
Average tip of  14  o'Clock is  1.23916575228452
Average tip of  15  o'Clock is  1.258098961464723
Average tip of  16  o'Clock is  1.3465213173172041
Average tip of  17  o'Clock is  1.3224847696402944
Average tip of  18  o'Clock is  1.3222653005078675
Average tip of  19  o'Clock is  1.348410626018